In [ ]:
# pip install --upgrade openai
import os
from openai import OpenAI

# Set OPENAI_API_KEY in your environment first
client = OpenAI(api_key='xxx')

In [3]:
def deep_research(query: str, *,
                  model: str = "o3-deep-research-2025-06-26",
                  summary: str = "auto"):
    """
    Run an open-web Deep Research task and return the report text + citations.
    Models: o4-mini-deep-research-2025-06-26 (faster/cheaper) or
            o3-deep-research-2025-06-26 (highest quality).
    """
    system_message = (
        "You are a professional researcher. Produce a structured, "
        "citation-rich report with concise sections. Prefer recent, "
        "authoritative sources. Include inline citations."
    )

    resp = client.responses.create(
        model=model,
        input=[
            {"role": "developer", "content": [{"type": "input_text", "text": system_message}]},
            {"role": "user", "content": [{"type": "input_text", "text": query}]},
        ],
        reasoning={"summary": summary},
        tools=[
            {"type": "web_search_preview"},          # allow open-web search
            # Optional: also let it run code for light analysis/plots
            {"type": "code_interpreter", "container": {"type": "auto", "file_ids": []}},
        ],
        # Tip: for very long jobs you can add background=True and poll (see below).
    )

    # Final report text
    report_text = resp.output[-1].content[0].text

    # Extract (title, url, span) for each inline citation
    annotations = getattr(resp.output[-1].content[0], "annotations", []) or []
    citations = [{"title": a.title, "url": a.url,
                  "span": (a.start_index, a.end_index)} for a in annotations]

    return report_text, citations

if __name__ == "__main__":
    text, cites = deep_research("Compare the top 3 open-source vector databases as of 2025. "
                                "Focus on performance, ecosystem, and TCO. Provide links.")
    print(text)
    print("\nCitations:")
    for c in cites:
        print(f"- {c['title']} — {c['url']}")


APIConnectionError: Connection error.